In [ ]:
import csv
import random
import pandas as pd
from datasets import Dataset, DatasetDict
import random
random.seed(1111)


In [ ]:
!pip install datasets

# Load and Process YouTube Comments

In [ ]:

# Initializing lists for storing comments and responses
comment_list = []

# Opening the CSV file containing YouTube comments
with open("comments_raw.csv", mode='r') as file:
    csv_reader = csv.DictReader(file)

    # Initializing a variable to keep track of the previous comment
    previous_comment = None

    # Reading the file line by line
    for row in csv_reader:
        # Extracting comments, authors, and reply status from each row
        comment = row['simpleText']
        author = row['author']
        is_reply = row['isReply']

        # Checking if the row is a non-reply comment
        if is_reply == 'FALSE':
            # Saving the previous comment if it's not a reply
            previous_comment = {'comment': comment, 'response': ''}
            comment_list.append(previous_comment)
        # Checking if the row is a reply by @LunarTech_ai
        elif is_reply == 'TRUE' and author == '@LunarTech_ai' and previous_comment:
            # If it's a reply by @LunarTech_ai and there is a previous comment, matching it
            previous_comment['response'] = comment
            # Resetting the previous comment after matching
            previous_comment = None

# Displaying the processed comments and responses
for entry in comment_list:
    print(f"Comment: {entry['comment']}")
    if entry['response']:
        print(f"Response: {entry['response']} -LunarTechGPT")
    else:
        print("Response: None")
    print("---")


Comment: Thank you for this..... After failing Maths in the 1970s, channels like yours have helped me to "have a go"—not for a career but to prevent cognitive decline. I may not understand it all, but that's okay; for me, just trying is important.
Response: That’s really amazing to hear Tx! -LunarTechGPT
---
Comment: in india we learn this in 11th and 12th standard
Response: None
---
Comment: That's what I was looking for, thank you.
Response: None
---
Comment: If I had a mathemtics teacher like that , I would love to do the class , even on Sunday.
Response: None
---
Comment: Thank you very much for this crash course!
Response: None
---
Comment: Excellent class.  Great course.
Response: Thank you for your feedback 🙌🏻 Mario -LunarTechGPT
---
Comment: Lovely tutorial, It would be even more interesting if the tutor was actually showing on the side of the video the whole time, So to make things more interactive. 😇
Response: None
---
Comment: Great video.
Response: None
---
Comment: Thank y

# Generating Formatted Examples for LunarTech

In [ ]:

# Instruction string for LunarTech
instructions_string = f"""LunarTech, functioning as an essential Data Science and AI consultant and developer on YouTube, communicates in clear, accessible language. \
It escalates to technical depth upon request and adds personal comments to make interactions engaging. \
LunarTech responds aptly to feedback and ends responses with its signature '–LunarTechGPT'. \
LunarTech tailors the length and complexity of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
and more detailed explanations where necessary. This ensures the interaction remains natural, engaging, and informative.

Please respond to the following comment:
"""

# Template function to format examples
example_template = lambda comment, response: f'''<s>[INST] {instructions_string} \n{comment} \n[/INST]\n''' + (response if response else "No response yet") + "</s>"

# Generating examples
example_list = [example_template(entry['comment'], entry['response']) for entry in comment_list]

# Displaying all generated examples
for example in example_list:
    print(example)

# Creating train/test split
# Calculate the number of test examples (20% of the total examples)
test_size = int(len(example_list) * 0.2)

# Select random indices for the test set
test_indices = random.sample(range(len(example_list)), test_size)

# Create the test list by selecting examples at the chosen indices
test_list = [example_list[i] for i in test_indices]

# Remove the test examples from the example list to create the training set
train_list = [example_list[i] for i in range(len(example_list)) if i not in test_indices]

# Creating a DatasetDict with train and test splits
data = DatasetDict({
    'train': Dataset.from_dict({"example": train_list}),
    'test': Dataset.from_dict({"example": test_list})
})

# Displaying the train and test datasets
print(data)

<s>[INST] LunarTech, functioning as an essential Data Science and AI consultant and developer on YouTube, communicates in clear, accessible language. It escalates to technical depth upon request and adds personal comments to make interactions engaging. LunarTech responds aptly to feedback and ends responses with its signature '–LunarTechGPT'. LunarTech tailors the length and complexity of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, and more detailed explanations where necessary. This ensures the interaction remains natural, engaging, and informative.

Please respond to the following comment:
 
Thank you for this..... After failing Maths in the 1970s, channels like yours have helped me to "have a go"—not for a career but to prevent cognitive decline. I may not understand it all, but that's okay; for me, just trying is important. 
[/INST]
That’s really amazing to hear Tx!</s>
<s>[INST] LunarTech, functionin

In [ ]:

# create train/test split
test_index_list = random.sample(range(0, len(example_list)-1), 9)

test_list = [example_list[index] for index in test_index_list]

for example in test_list:
    example_list.remove(example)



data = DatasetDict({'train':Dataset.from_dict({"example":example_list}), 'test':Dataset.from_dict({"example":test_list})})
print(data)



DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 143
    })
    test: Dataset({
        features: ['example'],
        num_rows: 9
    })
})


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
data.push_to_hub("TatevK/lunartech-youtube-comments")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/TatevK/lunartech-youtube-comments/commit/5dea427c664309ba99ff10e1d410b7ab3f17ffd5', commit_message='Upload dataset', commit_description='', oid='5dea427c664309ba99ff10e1d410b7ab3f17ffd5', pr_url=None, pr_revision=None, pr_num=None)